# Scrape Table Data for U.S. Air Force Bases and Installations for Active Duty, National Guard, and Reserve from Wikipedia

In [ ]:
# Import Dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import json
import re
from pprint import pprint

In [ ]:
# Launch browser
browser = Browser ('chrome')

## Step 1: Visit the Website

In [ ]:
# Visit the website
url = 'https://en.wikipedia.org/wiki/List_of_United_States_Air_Force_installations'
browser.visit(url)

## Step 2: Scrape the Website

In [ ]:
# Create Beautiful Soup
html = browser.html
soup = bs(html, 'html.parser')
tables = soup.find_all('table', class_='wikitable sortable jquery-tablesorter')

## Step 3: Store the Results

In [ ]:
# Initialize an empty list to hold data from both tables
all_data = []

In [ ]:
# Standardize header names
def standardize_header_name(header):
    return header.replace(' ', '_').replace('-', '_').lower()

In [ ]:
# Process each table and append the data
for index, table in enumerate(tables[:2]): # Limit to the first two tables
    
    # Extract header row
    header_row = table.find('tr')
    headers = [standardize_header_name(th.text.strip()) for th in header_row.find_all('th')]
    
    # Extract data rows and append to all_data with a table identifier
    rows = []
    for tr in table.find_all('tr')[1:]:  # skip the first row as it contains headers
        cells = [td.get_text(strip=True) for td in tr.find_all('td')]
        if cells:
            rows.append(cells)

    # Save each table to DataFrame then append to all_data list
    df = pd.DataFrame(rows, columns=headers)
    all_data.append(df)

In [ ]:
# Verify table data for Table 1 - AFB, Active Duty Locations
active_duty_df = all_data[0]
active_duty_df

In [ ]:
# Verify table data for Table 2 - AFB, National Guard and Reserve Locations
reserve_df = all_data[1]
reserve_df

In [ ]:
# Rename 'state' column without reassigning to a new variable
reserve_df.rename(columns={'state': 'state_or_area'}, inplace=True)
reserve_df

In [ ]:
# Verify that column headers match in both tables
print("Active Duty DF Columns:", active_duty_df.columns)
print("Reserve DF Columns:", reserve_df.columns)

In [ ]:
# Concatenate the DataFrames
combined_df = pd.concat([active_duty_df, reserve_df], ignore_index=True)
combined_df

## Step 4: Clean the DataFrame

### Step 4a: Split the 'Coordinates' column into two new columns

In [ ]:
# Split the 'coordinates' column into two new columns
split_df = combined_df['coordinates'].str.split(' / ', expand=True)

# Ensure there are always two columns
if split_df.shape[1] == 1:
    
    # Add a second column with NaN if only one column resulted from the split
    split_df[1] = pd.NA

# Assign split columns to the original DataFrame
combined_df['dms_coordinates'] = split_df[0]
combined_df['decimal_coordinates'] = split_df[1]

# Print the DataFrame to verify the output
print(combined_df[['dms_coordinates', 'decimal_coordinates']])

In [ ]:
# Increase the maximum width of the column to, say, 1000 characters
pd.set_option('display.max_colwidth', 1000)

coordinates = combined_df['coordinates']
coordinates

In [ ]:
# Split the 'coordinates' column into potentially three parts
split_columns = combined_df['coordinates'].str.split(' / ', expand=True)

# Assign these new columns back to the original DataFrame
combined_df['dms_coordinates'] = split_columns[0]

# Safely assign second part if it exists
combined_df['decimal_coordinates'] = split_columns[1] if split_columns.shape[1] > 1 else pd.NA

# Safely assign third part if it exists
combined_df['geojson_coordinates'] = split_columns[2] if split_columns.shape[1] > 2 else pd.NA

In [ ]:
combined_df

### Step 4b: Extract just the numeral from 'decimal_coordinates' column

In [ ]:
# Clean and convert the 'decimal_coordinates' column by extracting just the numeral
def clean_and_format_coords(coord):
    
    if pd.isna(coord):
        return None  # Handle missing values
    
    # Extract numbers and potential N, S, E, W characters
    parts = re.findall(r'([+-]?\d+\.\d+)([NSEW]?)', coord)
    
    if not parts:
        return None
    
    # Prepare latitude and longitude, considering the direction
    cleaned_parts = []
    
    for value, direction in parts:
        num = float(value)

        # South or West should be negative
        if direction == 'S' or direction == 'W':
            num = -num
        cleaned_parts.append(num)
    
    if len(cleaned_parts) == 2:
        # Ensure longitude comes first for GeoJSON
        return (cleaned_parts[1], cleaned_parts[0])
    return None

# Apply this cleaning and formatting to the DataFrame
combined_df['geojson_coordinates'] = combined_df['decimal_coordinates'].apply(clean_and_format_coords)
combined_df

### Step 4c: Split the newly created 'geojson_coordinates' column into separate 'longitude' and 'latitude' columns

In [ ]:
# Split the 'geojson_coordinates' into 'longitude' and 'latitude'
combined_df['longitude'], combined_df['latitude'] = zip(*combined_df['geojson_coordinates'])

# Print the updated DataFrame to check the new columns
print(combined_df[['longitude', 'latitude']])

## Step 5: Clean the DataFrame by dropping unnecessary columns, such as all the 'coordinates' and the 'emblem' columns

In [ ]:
# Drop the columns that are no longer needed
combined_df = active_duty_df.drop(columns=[
    'coordinates', 
    'wing_or_unit_emblem', 
    'dms_coordinates', 
    'decimal_coordinates', 
    'geojson_coordinates'
])

combined_df

## Step 6: Save the DataFrame as a JSON file

In [ ]:
# Convert DataFrame to a JSON string
json_str = combined_df.to_json(orient='records')

# Parse the JSON string back into a Python list of dictionaries
data = json.loads(json_str)

# Write the JSON file with indentation for better readability
with open('air_force_base_us.json', 'w') as f:
    json.dump(data, f, indent=4)

# Optionally, print the JSON string to the console for verification
print(json.dumps(data, indent=4))

In [ ]:
browser.quit()